In [12]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import datetime

In [13]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csv
df = pd.read_csv("high_ratio_0.15_volume_money_30000000000_v2.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=4)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [14]:
fin_result

,interval_end,shcode,target_day,target_m,shname
0,20190826,000140,20190729,201907,하이트진로홀딩스
1,20190612,000890,20190515,201905,보해양조
2,20190410,001140,20190313,201903,카리스국보
3,20191015,001340,20190917,201909,백광산업
4,20190605,001550,20190508,201905,조비
5,20190614,001550,20190517,201905,조비
6,20190516,001745,20190418,201904,SK네트웍스우
7,20190813,001820,20190716,201907,삼화콘덴서
8,20190822,002070,20190725,201907,남영비비안
9,20190823,002070,20190726,201907,남영비비안


In [ ]:
#1. 이평 정배열 확인한다.

#3. 


In [54]:
#4. target month
#8월대상한정
#fin_result.groupby(['target_m']).count()
target_month = '201907'
target_fin_result = fin_result[fin_result['target_m'] == target_month]
target_fin_result = target_fin_result.reset_index()

In [57]:
target_fin_result

,index,interval_end,shcode,target_day,target_m,shname
0,0,20190826,000140,20190729,201907,하이트진로홀딩스
1,7,20190813,001820,20190716,201907,삼화콘덴서
2,8,20190822,002070,20190725,201907,남영비비안
3,9,20190823,002070,20190726,201907,남영비비안
4,32,20190801,004830,20190704,201907,덕성
5,38,20190805,005360,20190708,201907,모나미
6,39,20190805,005390,20190708,201907,신성통상
7,44,20190820,006890,20190723,201907,태경화학
8,58,20190822,008730,20190725,201907,율촌화학
9,79,20190823,012610,20190726,201907,경인양행


In [ ]:
'''
#1. 이평 정배열 확인한다.

# 대세 판단1 : 월봉 20이평이 증가 중 12개?
# 대세 판단2 : 월봉 5이평이 증가 중
# 대세 판단3 : 월봉 ma5 - ma20  크눈중

#대세 판단 비고 : 외인 비율 확인하기
# 월봉,주봉 이평정배열(이평 5가 20보다 위인 any) or 
# 눌림목 발생하고 ,  거래량 똥 망하고,
# range 사이의 어디구간 30%~50%(구분지어서볼것), 
'''
'''
#2. 시점기준 관련된 주체 확인하여 쿼리로 작성한다. 최근관련된정보 한해서 확인(7,8 등)

'''

In [ ]:
fin_result

In [ ]:
fin_result['target_day']

In [15]:
i = 0

In [51]:
df.head()

,date,open,high,low,close,volume
0,20190729,9310,12000,9260,12000,6291621
1,20190730,12350,14150,12000,12050,7616938
2,20190731,12150,12300,10450,10650,2326322
3,20190801,10650,11100,9820,9950,1521007
4,20190802,10400,10850,9970,10400,2206395


In [43]:
df = query.query_ohlcv(shcode = fin_result['shcode'].iloc[i], 
                      fr = fin_result['target_day'].iloc[i], 
                      to = fin_result['interval_end'].iloc[i])
open_trigger = df['open'].iloc[0]
close_trigger = df['close'].iloc[0]
#df['close']>=
ratio_list = pd.DataFrame(dict(close_ratio = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])],
                              date = np.array(df)[:,0][1:]))

In [52]:
# daily open <= open
open_touch_list = df['date'].iloc[1:][df['open'] <= open_trigger]
# daily close >= close
close_touch_list = df['date'].iloc[1:][df['close'] >= close_trigger]

In [53]:
close_touch_list

1    20190730
5    20190805
6    20190806
Name: date, dtype: object

In [36]:
pd.DataFrame([[y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])],[3 for i in range(19)]] )

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.004167,-0.116183,-0.065728,0.045226,0.173077,0.012295,-0.064777,-0.051948,-0.009132,0.009217,-0.03653,0.0,-0.042654,0.00495,-0.004926,0.019802,0.0,0.029126,-0.028302
1,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.00000,3.0,3.000000,3.00000,3.000000,3.000000,3.0,3.000000,3.000000


In [40]:
ratio_list = pd.DataFrame(dict(close_ratio = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])],
                              date = np.array(df)[:,0][1:]))

In [41]:
result

,close_ratio,date
0,0.004167,20190730
1,-0.116183,20190731
2,-0.065728,20190801
3,0.045226,20190802
4,0.173077,20190805
5,0.012295,20190806
6,-0.064777,20190807
7,-0.051948,20190808
8,-0.009132,20190809
9,0.009217,20190812


In [30]:
12200/ 10400-1

0.17307692307692313

In [19]:
df

,date,open,high,low,close,volume
0,20190729,9310,12000,9260,12000,6291621
1,20190730,12350,14150,12000,12050,7616938
2,20190731,12150,12300,10450,10650,2326322
3,20190801,10650,11100,9820,9950,1521007
4,20190802,10400,10850,9970,10400,2206395
5,20190805,10900,12900,10450,12200,5418072
6,20190806,11600,13450,11500,12350,3341082
7,20190807,12850,12900,11250,11550,1821572
8,20190808,11300,11300,10450,10950,1110817
9,20190809,11100,11100,10650,10850,528349


In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1
            
    return average_volume_test

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')